In [1]:
import glob 
import os
from tqdm import tqdm

covers = []

In [2]:
from pathlib import Path

for path in tqdm(Path('/volumes/Multimedia/FLAC/').rglob('*.jpg')):
    absolute = os.path.abspath(path)
    covers.append(absolute)

0it [00:00, ?it/s]


In [3]:
#len(os.listdir('/volumes/Multimedia/FLAC/'))

# Setup

In [4]:
import requests

#!pip install musicbrainzngs
#!pip install pyvis




In [5]:
import musicbrainzngs

In [6]:
auth = musicbrainzngs.auth("username","password")

In [7]:
(musicbrainzngs.set_useragent("app","Client_ID", 
                              contact="email"))

# Get Results for Artist ID Search

In [8]:
import pandas as pd

In [9]:
#Need to modify to label the purpose: Producer, Performer, etc

def get_artist_works(artist_id):

    result_artist = (musicbrainzngs.get_artist_by_id(artist_id
                                                     ,includes=['work-rels']))

    df_artist_work_relation = pd.DataFrame()

    df_artist_interate = pd.DataFrame(result_artist['artist']['work-relation-list'])

    #Work ID - Loop by length of dataframe - work[0] on outer to get id
    for x in range(len(df_artist_interate)):
        df_iter = pd.DataFrame(result_artist['artist']['work-relation-list'][x])
        df_artist_work_relation = pd.concat([df_artist_work_relation,df_iter])
        
    
    product = df_artist_work_relation.reset_index()
    
    artist_work_id = product[product['index']=="id"]['work'].to_list()
    artist_work_name = product[product['index']=="title"]['work'].to_list()
    
    
    
    return artist_work_id, artist_work_name, product

In [10]:
David_Byrne = 'd4659efb-b8eb-4f03-95e9-f69ce35967a9'



In [11]:
work_ids,work_names,_ = get_artist_works(David_Byrne)

In [130]:
from tqdm.autonotebook import tqdm
def establish_work_to_artist_relationships(work_ids,work_names):
    
    artist_names = []
    
    for index, ident in enumerate(work_ids):
    
        result_work = (pd.DataFrame(musicbrainzngs.get_work_by_id(ident
                                                                  ,includes=['artist-rels'])))

        works = pd.DataFrame(result_work['work']['artist-relation-list'])
       
        artists = []
        
        
        for x in range(len(works)):

            artists.append(works.iloc[x]['artist']['name'])
            
        

    
        artist_names.append({f'{work_names[index]}':artists})
    
    return artist_names
    

    #pd.DataFrame(result_work['work']['artist-relation-list'][5])#['artist']['id']
    #Artist ID  - Loop by length of dataframe - artist[0] on outer to get id



In [131]:
a = establish_work_to_artist_relationships(work_ids,work_names)

In [132]:
a

[{'Lazy': ['Ashley Beedle',
   'David Byrne',
   'Darren House',
   'Rocky',
   'Ashley Beedle',
   'David Byrne',
   'Darren House',
   'Rocky']},
 {'Paper Emperor': ['David Byrne', '[traditional]']},
 {'The Last Emperor': ['David Byrne', 'Cong Su', '坂本龍一']},
 {'(Nothing but) Flowers': ['David Byrne',
   'Chris Frantz',
   'Jerry Harrison',
   'Tina Weymouth',
   'David Byrne']},
 {"A Clean Break (Let's Work)": ['David Byrne']},
 {'Adé': ['David Byrne', 'Brian Eno']},
 {'Admiral Perry': ['David Byrne']},
 {'Air': ['David Byrne', 'David Byrne']},
 {'And She Was': ['David Byrne', 'David Byrne']},
 {'Animals': ['David Byrne', 'David Byrne']},
 {'Artists Only': ['David Byrne', 'Wayne Zieve']},
 {'Bed': ['David Byrne']},
 {'Big Blue Plymouth (Eyes Wide Open)': ['David Byrne', 'David Byrne']},
 {'Big Daddy': ['David Byrne',
   'Chris Frantz',
   'Jerry Harrison',
   'Tina Weymouth',
   'David Byrne']},
 {'Bill': ['David Byrne',
   'Chris Frantz',
   'Jerry Harrison',
   'Tina Weymouth',
   

In [133]:
#b = pd.DataFrame(a)

#artist_names = b['Artist_Name'].to_list()



In [158]:
#Visualize and takes care of duplicates for you
#Add nodes and edges properly

from pyvis.network import Network

# importing networkx  
import networkx as nx
from networkx import algorithms
from networkx import drawing,layout
# importing matplotlib.pyplot 
import matplotlib.pyplot as plt 

def visualize_graph(starter,connections1,connections2,yes=True):

    
    g = nx.Graph(scale=0.01,bgcolor="#222222")
    pos = nx.spring_layout(g)
    g.add_node(starter)

    for node in connections1:

        g.add_node(node,title=node)
        g.add_node('David Byrne',title=starter)
        
        
        g.add_edge(node,starter)
    
    for x in connections2:
        for key, values in x.items():
            print(key,values)
            for branch in values:
                g.add_node(branch,title=branch)
                g.add_edge(key,branch)
            
            
        g.add_nodes_from([
        (node, {"color": "red"}),
        (branch, {"color": "green"}),
        (starter, {"color": "yellow"}),
        (key, {"color": "blue"})])
    
    nt = Network("2000px", "2000px")
    # populates the nodes and edges data structures
    layout.spring_layout(g)
    nt.from_nx(g)
    nt.show("nx.html")
    
    return g
    #plt.savefig("filename.png")
    
    
    
    g = nx.Graph() 

    #nx_graph = Network()
    #nx_graph.nodes[1]['title'] = 'Number 1'
    #nx_graph.nodes[1]['group'] = 1
    #nx_graph.nodes[3]['title'] = 'I belong to a different group!'
    #nx_graph.nodes[3]['group'] = 10
    #nx_graph.add_nodes(connections1)
    #nx_graph.add_node(21, size=15, title='couple', group=2)
    #nx_graph.add_edges(connections1)
    #nx_graph.add_node(25, size=25, label='lonely', title='lonely node', group=3)
    #nt = Network("500px", "500px")
    # populates the nodes and edges data structures
    #nt.from_nx(nx_graph)
    #nt.show("nx.html")


In [159]:
g = visualize_graph(David_Byrne,work_names[:50],a,yes=False)

Lazy ['Ashley Beedle', 'David Byrne', 'Darren House', 'Rocky', 'Ashley Beedle', 'David Byrne', 'Darren House', 'Rocky']
Paper Emperor ['David Byrne', '[traditional]']
The Last Emperor ['David Byrne', 'Cong Su', '坂本龍一']
(Nothing but) Flowers ['David Byrne', 'Chris Frantz', 'Jerry Harrison', 'Tina Weymouth', 'David Byrne']
A Clean Break (Let's Work) ['David Byrne']
Adé ['David Byrne', 'Brian Eno']
Admiral Perry ['David Byrne']
Air ['David Byrne', 'David Byrne']
And She Was ['David Byrne', 'David Byrne']
Animals ['David Byrne', 'David Byrne']
Artists Only ['David Byrne', 'Wayne Zieve']
Bed ['David Byrne']
Big Blue Plymouth (Eyes Wide Open) ['David Byrne', 'David Byrne']
Big Daddy ['David Byrne', 'Chris Frantz', 'Jerry Harrison', 'Tina Weymouth', 'David Byrne']
Bill ['David Byrne', 'Chris Frantz', 'Jerry Harrison', 'Tina Weymouth', 'David Byrne']
Black Flag ['David Byrne']
Blind ['David Byrne', 'Chris Frantz', 'Jerry Harrison', 'Tina Weymouth', 'David Byrne']
Born Under Punches (The Heat G

In [154]:
from pyvis.network import Network
import pandas as pd

got_net = Network(height="750px", width="100%", bgcolor="#222222", font_color="white")

# set the physics layout of the network
got_net.barnes_hut()
got_data = pd.read_csv("https://www.macalester.edu/~abeverid/data/stormofswords.csv")

sources = got_data['Source']
targets = got_data['Target']
weights = got_data['Weight']



edge_data = zip(sources, targets, weights)

for e in edge_data:
    src = e[0]
    dst = e[1]
    w = e[2]

    got_net.add_node(src, src, title=src)
    got_net.add_node(dst, dst, title=dst)
    got_net.add_edge(src, dst, value=w)

neighbor_map = got_net.get_adj_list()

print(neighbor_map)

# add neighbor data to node hover data
for node in got_net.nodes:
    node["title"] += " Neighbors:<br>" + "<br>".join(neighbor_map[node["id"]])
    node["value"] = len(neighbor_map[node["id"]])

got_net.show("gameofthrones.html")

{'Aemon': {'Samwell', 'Robert', 'Stannis', 'Jon', 'Grenn'}, 'Grenn': {'Samwell', 'Aemon', 'Jon', 'Eddison'}, 'Samwell': {'Craster', 'Qhorin', 'Mance', 'Bran', 'Bowen', 'Jon', 'Melisandre', 'Stannis', 'Grenn', 'Eddison', 'Jojen', 'Aemon', 'Janos', 'Gilly', 'Meera'}, 'Aerys': {'Tyrion', 'Tywin', 'Jaime', 'Robert'}, 'Jaime': {'Loras', 'Elia', 'Cersei', 'Stannis', 'Balon', 'Tywin', 'Meryn', 'Aerys', 'Qyburn', 'Eddard', 'Catelyn', 'Tommen', 'Brienne', 'Gregor', 'Walton', 'Joffrey', 'Edmure', 'Barristan', 'Robert', 'Arya', 'Sansa', 'Robb', 'Tyrion', 'Renly'}, 'Robert': {'Tywin', 'Barristan', 'Aerys', 'Sandor', 'Jon Arryn', 'Daenerys', 'Eddard', 'Arya', 'Sansa', 'Cersei', 'Thoros', 'Jon', 'Stannis', 'Tyrion', 'Aemon', 'Jaime', 'Rhaegar', 'Renly'}, 'Tyrion': {'Loras', 'Elia', 'Cersei', 'Stannis', 'Jaime', 'Bronn', 'Balon', 'Tywin', 'Meryn', 'Aerys', 'Sandor', 'Viserys', 'Catelyn', 'Lysa', 'Kevan', 'Podrick', 'Doran', 'Janos', 'Ilyn', 'Chataya', 'Varys', 'Joffrey', 'Mace', 'Margaery', 'Shae', '

In [155]:
#Neighbors
for i in g.neighbors('David Byrne'):
    print(i)

Lazy
Paper Emperor
The Last Emperor
(Nothing but) Flowers
A Clean Break (Let's Work)
Adé
Admiral Perry
Air
And She Was
Animals
Artists Only
Bed
Big Blue Plymouth (Eyes Wide Open)
Big Daddy
Bill
Black Flag
Blind
Born Under Punches (The Heat Goes On)
Burning Down the House
Cities
City of Dreams
City of Steel
Cloud Chamber
Combat
Cool Water
Creatures of Love
Dance on Vaseline
Dense Beasts
Der Weg in die Ferne
Dinosaur
Don’t Worry About the Government
Double Groove
Dream Operator
Eggs in a Briar Patch
Fela's Riff (unfinished outtake)
First Week/Last Week…Carefree
Five Golden Sections
Found a Job
Girlfriend Is Better
Give Me Back My Name
Hanging Upside Down
Happy Day
Hey Now
High Life for Nine Instruments
His Wife Refused
Houses in Motion
I Feel It in My Heart
I Get Wild/Wild Gravity
I Wish You Wouldn't Say That
I Zimbra
I’m Not in Love
In the Future
Jungle Book
Knotty Pine
Lifetime Piling Up
Light Bath
Listening Wind
Love → Building on Fire
Love for Sale
Main Title Theme
Making Flippy Flop

In [ ]:
#Central Theme
nx.degree_centrality(g)

In [ ]:
#Shortest Path
centrality.betweenness_centrality(g)

# Artist Specific

In [ ]:
result = musicbrainzngs.get_artist_by_id(artist_id,
              includes=['release-groups'],release_type=['album'])


#for release_group in result["artist"]["release-group-list"]:
#    print("{title} ({type})".format(title=release_group["title"],
#                                    type=release_group["type"]))

In [ ]:
#result['artist']

In [ ]:
import pandas as pd
df = pd.DataFrame(result["artist"]["release-group-list"])
df["first-release-date"] = pd.to_datetime(df["first-release-date"])  

df["year"] = pd.DatetimeIndex(df["first-release-date"]).year
df['month'] = pd.DatetimeIndex(df["first-release-date"]).month
df["day"] = pd.DatetimeIndex(df["first-release-date"]).day

In [ ]:
df

In [ ]:
_ = df.groupby("year")["id"].count().plot(kind="bar",title="Annual: Artist Releases")

# Get Album Cover Art

In [ ]:
release_id = "0dd5a352-6ae0-3e0d-bf18-bec30e27807d"
#"7e643344-946e-4118-b934-6574695e68bf"
data = musicbrainzngs.get_release_group_image_list(release_id)


for image in data["images"]:
    if "Front" in image["types"] and image["approved"]:
        print("%s is an approved front image!" % image["thumbnails"]["large"])
        image_link = image["thumbnails"]["large"]
        break
        


# Track Specific

In [ ]:
#musicbrainzngs.get_release_group_by_id(release_id, includes=['releases'])

In [ ]:
#musicbrainzngs.get_release_by_id('f7345598-ef59-44e1-88d0-4b428977fda1',includes=['recordings'])

In [ ]:
#musicbrainzngs.get_recording_by_id('ae077045-d684-466d-823e-b6f229733a5b',includes=['artist-credits'])

In [ ]:
#response = requests.get(image_link, stream=True)
#with open('img.png', 'wb') as out_file:
#    shutil.copyfileobj(response.raw, out_file)